In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import pandas as pd
from sklearn.metrics import classification_report
import torch
print(torch.__version__)
print(torch.cuda.is_available())
from transformers.utils import is_torch_available, is_tf_available

print("Torch disponible:", is_torch_available())
print("TensorFlow disponible:", is_tf_available())

c:\Users\migue\Documents\GitHub\Practica_BAIN_4\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.5.1+cu121
True
Torch disponible: True
TensorFlow disponible: False


In [2]:
df = pd.read_csv('INPUT/emociones.csv')
df = df[['opinion', 'label']]
df = df.dropna()
df = df.sample(30, random_state=42).reset_index(drop=True)
print(df)


                       opinion     label
0          Es un gran servicio  positiva
1                   Es regular   neutral
2          Es un producto útil  positiva
3                 Es un fraude  negativa
4               Me gusta mucho  positiva
5               Es un desastre  negativa
6           Es un mal servicio  negativa
7        Es un producto inútil  negativa
8           No lo recomendaría  negativa
9       Me encantó el producto  positiva
10                 No me gusta  negativa
11               Es fantástico  positiva
12                  Me encanta  positiva
13         Es un buen producto  positiva
14                Es muy bueno  positiva
15          Es un mal servicio  negativa
16    Es una pérdida de dinero  negativa
17         No es nada especial   neutral
18       Es un producto normal   neutral
19                     Te odio  negativa
20         Es un buen servicio  positiva
21         Es un gran producto  positiva
22      Es un producto decente   neutral
23  Es un produc

In [3]:
# Load model and tokenizer
model_id = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Create the pipeline
classifier = pipeline('text2text-generation', model=model, tokenizer=tokenizer)

# Lista para almacenar las predicciones
predictions_base = []
predictions_zero_shot = []
predictions_few_shot = []
predictions_cot = []

# Clasificar cada opinión y mostrar resultado
for row in df.itertuples():
    opinion = row.opinion
    label = row.label

    prompt_base = f"Please classify the following opinion based on the tone of the message. The tone can be positivo, negativo, or neutral: {opinion}"


    zero_shot_prompt = f"Classify the sentiment of the following opinion as either positivo, negativo, or neutral: {opinion}"



    few_shot_prompt = f"""
    Aquí tienes algunas opiniones clasificadas como ejemplo:

    Opinión: '¡Me encanta este producto! Es increíble y ha superado todas mis expectativas. Definitivamente lo recomendaré.' → Sentimiento: Positivo  
    Opinión: 'El servicio fue terrible. Tuve que esperar mucho y el personal fue muy poco amable. No lo recomendaría a nadie.' → Sentimiento: Negativo  
    Opinión: 'El producto está bien, pero no es nada excepcional. Cumple su función, aunque esperaba algo más por el precio.' → Sentimiento: Neutral  
    Opinión: 'Es el mejor libro que he leído. Me mantuvo enganchado todo el tiempo. ¡Lo recomendaré a todos mis amigos!' → Sentimiento: Positivo  
    Opinión: 'La comida estuvo fría y no sabía tan bien como esperaba. No volveré a pedir aquí.' → Sentimiento: Negativo  
    Opinión: 'La película fue entretenida, pero no me dejó una impresión duradera. Estuvo bien para pasar el rato.' → Sentimiento: Neutral  

    Ahora, clasifica el sentimiento de la siguiente opinión:

    Opinión: {opinion}
    """



    cot_prompt = f"""
    Carefully analyze the following opinion:

    Opinion: {opinion}

    The sentiment can be classified as positive, negative, or neutral. Consider the tone, language, and context of the opinion. What is your reasoning for classifying it in this way? Please explain your thought process step by step.
    """




    prompts = {
        "prompt": prompt_base,
        "zero-shot": zero_shot_prompt,
        "few-shot": few_shot_prompt,
        "chain-of-thought": cot_prompt
    }

    # Generar predicción con Zero-shot
    prompt_zero_shot = zero_shot_prompt.format(opinion)
    result_zero_shot = classifier(prompt_zero_shot)
    sentiment_zero_shot = result_zero_shot[0]["generated_text"]
    
    
    # Generar predicción con Few-shot
    prompt_few_shot = few_shot_prompt.format(opinion)
    result_few_shot = classifier(prompt_few_shot, max_length=100)
    sentiment_few_shot = result_few_shot[0]["generated_text"]

    # Generar predicción con el prompt base
    prompt = prompt_base.format(opinion)
    result_base = classifier(prompt)
    sentiment_base = result_base[0]["generated_text"]
    
    # Generar predicción con Chain-of-thought
    prompt_cot = cot_prompt.format(opinion)
    result_cot = classifier(prompt_cot, max_length=100)
    sentiment_cot = result_cot[0]["generated_text"]
    
    
    # Asignar sentimientos de acuerdo al texto generado
    def assign_sentiment(response):
        normalized_response = response.lower().strip()

        palabras_positivas = ["positivo", "positiva", "positive"]
        palabras_negativas = ["negativo", "negativa", "negative"]
        palabras_neutras = ["neutral", "neutro", "neutra"]

        if any(word in normalized_response for word in palabras_positivas):
            return "positiva"
        elif any(word in normalized_response for word in palabras_negativas):
            return "negativa"
        elif any(word in normalized_response for word in palabras_neutras):
            return "neutral"
        else:
            return "desconocida"
    
    # Asignar sentimientos a cada predicción
    predictions_base.append(assign_sentiment(sentiment_base))
    predictions_zero_shot.append(assign_sentiment(sentiment_zero_shot))
    predictions_few_shot.append(assign_sentiment(sentiment_few_shot))
    predictions_cot.append(assign_sentiment(sentiment_cot))

    # Imprimir resultados
    print(f"Opinión: {opinion} - Label: {label}\n → Base-Sentimiento: Sentimiento: {predictions_base[-1]}")
    print(f" → Zero-shot Sentimiento: {predictions_zero_shot[-1]}")
    print(f" → Few-shot Sentimiento: {predictions_few_shot[-1]}")
    print(f" → Chain-of-thought Sentimiento: {predictions_cot[-1]}\n")
    

# Añadir las predicciones al DataFrame
df['prompt_base'] = prompt_base
df['prediccion_zero_shot'] = predictions_zero_shot
df['prediccion_few_shot'] = predictions_few_shot
df['prediccion_cot'] = predictions_cot

# Mostrar el DataFrame con los resultados
print(df[['opinion', 'label', 'prompt_base' ,'prediccion_zero_shot', 'prediccion_few_shot', 'prediccion_cot']])

Device set to use cuda:0


Opinión: Es un gran servicio - Label: positiva
 → Base-Sentimiento: Sentimiento: positiva
 → Zero-shot Sentimiento: positiva
 → Few-shot Sentimiento: neutral
 → Chain-of-thought Sentimiento: positiva

Opinión: Es regular - Label: neutral
 → Base-Sentimiento: Sentimiento: neutral
 → Zero-shot Sentimiento: positiva
 → Few-shot Sentimiento: neutral
 → Chain-of-thought Sentimiento: negativa



You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Opinión: Es un producto útil - Label: positiva
 → Base-Sentimiento: Sentimiento: neutral
 → Zero-shot Sentimiento: positiva
 → Few-shot Sentimiento: neutral
 → Chain-of-thought Sentimiento: negativa

Opinión: Es un fraude - Label: negativa
 → Base-Sentimiento: Sentimiento: neutral
 → Zero-shot Sentimiento: negativa
 → Few-shot Sentimiento: neutral
 → Chain-of-thought Sentimiento: negativa

Opinión: Me gusta mucho - Label: positiva
 → Base-Sentimiento: Sentimiento: positiva
 → Zero-shot Sentimiento: positiva
 → Few-shot Sentimiento: neutral
 → Chain-of-thought Sentimiento: negativa

Opinión: Es un desastre - Label: negativa
 → Base-Sentimiento: Sentimiento: negativa
 → Zero-shot Sentimiento: negativa
 → Few-shot Sentimiento: neutral
 → Chain-of-thought Sentimiento: negativa

Opinión: Es un mal servicio - Label: negativa
 → Base-Sentimiento: Sentimiento: negativa
 → Zero-shot Sentimiento: negativa
 → Few-shot Sentimiento: neutral
 → Chain-of-thought Sentimiento: negativa

Opinión: Es un 

**Análisis de aciertos por prompt**

In [4]:
aciertos_df = pd.DataFrame({
    'prompt_base': predictions_base,
    'zero_shot': predictions_zero_shot,
    'few_shot': predictions_few_shot,
    'chain_of_thought': predictions_cot
})

aciertos = {
    'prompt_base': (aciertos_df['prompt_base'] == df['label']).sum(),
    'zero_shot': (aciertos_df['zero_shot'] == df['label']).sum(),
    'few_shot': (aciertos_df['few_shot'] == df['label']).sum(),
    'chain_of_thought': (aciertos_df['chain_of_thought'] == df['label']).sum()
}

print(f"Aciertos con la prompt base: {aciertos['prompt_base']} - Ratio: {aciertos['prompt_base'] / len(aciertos_df):.2f}")
print(f"Aciertos con la prompt zero shot: {aciertos['zero_shot']} - Ratio: {aciertos['zero_shot'] / len(aciertos_df):.2f}")
print(f"Aciertos con la prompt few shots: {aciertos['few_shot']}- Ratio: {aciertos['few_shot'] / len(aciertos_df):.2f}")
print(f"Aciertos con la prompt chain of thought: {aciertos['chain_of_thought']} - Ratio: {aciertos['chain_of_thought'] / len(aciertos_df):.2f}")

Aciertos con la prompt base: 20 - Ratio: 0.67
Aciertos con la prompt zero shot: 22 - Ratio: 0.73
Aciertos con la prompt few shots: 5- Ratio: 0.17
Aciertos con la prompt chain of thought: 17 - Ratio: 0.57
